In [ ]:
def get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i):
    f1 = bit_Flips[i][ix]
    f2 = bit_Flips_LH[i][ix] / bit_Flips[i][ix+1]
    f3 = bit_Flips_LH[i][ix] / bit_Flips[i][ix]
    f4 = bit_Flip_LH_nos[i][ix] / (1 - bit_Flips[i][ix+1])
    f5 = bit_Flip_LH_nos[i][ix] / (1 - bit_Flips[i][ix])
    return np.array([f1,f2,f3,f4,f5])
def get_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i):
    cur_bit_feature = []
    feature_sub1 = get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix, i)
    feature_sub2 = get_sub_feature(bit_Flips, bit_Flips_LH, bit_Flip_LH_nos, ix+1, i)
    feature_sub3 = feature_sub2 - feature_sub1
    return np.concatenate((feature_sub1,feature_sub2,feature_sub3))
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
model_msb = RandomForestClassifier(n_estimators= 30)
model_msb.fit(traindatas_msb, trainlabels_msb)
param_distribution = [
    {'n_estimators' : [90,150,200,250,300], 'max_features' : [1, 2, 3, 4, 5, 6], 
     'max_depth':[1,2,3,4,5],'min_samples_leaf':[3,4,5,6]}
]
 
from sklearn.model_selection import RandomizedSearchCV 
random_search_cv = RandomizedSearchCV(model_msb,
                                      param_distribution,
                                      n_iter = 50,
                                      cv = 10,
                                      scoring='neg_mean_squared_error',
                                      n_jobs = 1)
random_search_cv.fit(tun_data, tun_label) 
model_tunning = random_search_cv.best_estimator_


In [ ]:
def get_matrix_info():
    res = []
    for message in allDatas:
        matrix = [[0] * 8 for _ in range(8)]
        del message[0]
        deep = 0
        for signal in message:
            deep = deep + 1
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            if endianness == 0:
                count = 0
                x = start_row
                y = start_col
                while count < length:
                    cur_bit = [7-x,y,deep]
                    matrix[x][y] = cur_bit
                    if y == 7:
                        y = 0
                        x = x + 1
                    else:
                        y = y + 1
                    count = count + 1
            elif endianness == 1:
                count = 0
                x = start_row
                y = start_col
                while count < length:
                    cur_bit = [7-x,y,deep]
                    matrix[x][y] = cur_bit
                    if y == 0:
                        y = 7
                        x = x + 1
                    else:
                        y = y - 1
                    count = count + 1
        for i in range(8):
            for j in range(8):
                if matrix[i][j] == 0:
                    matrix[i][j] = [7-i,j,0]
        res.append(matrix)
    return res

In [ ]:
def get_matrix_info():
    res = []
    for message in allDatas:
        matrix = [[0] * 8 for _ in range(8)]
        del message[0]
        deep = 0
        for signal in message:
            deep = deep + 1
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            count = 0
            x = start_row
            y = start_col
            while count < length:
                cur_bit = [7-x,y,deep]
                matrix[x][y] = cur_bit
                if endianness == 0:
                    if y == 7:
                        y = 0
                        x = x + 1
                    else:
                        y = y + 1
                else:
                    if y == 0:
                        y = 7
                        x = x + 1
                    else:
                        y = y - 1
                count = count + 1
        for i in range(8):
            for j in range(8):
                if matrix[i][j] == 0:
                    matrix[i][j] = [7-i,j,0]
        res.append(matrix)
    return np.concatenate(res,axis=0)